### 5. Transfer Learning
- Evaluate the accuracy of your model on a pre-trained models like ImagNet, VGG16, Inception... (pick one an justify your choice)
- Perform transfer learning with your chosen pre-trained models i.e., you will probably try a few and choose the best one.

In [37]:
# General
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tabulate import tabulate
import cv2
from sklearn.model_selection import train_test_split

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torchvision.models import ResNet, resnet50
from torchvision import models


# TensorFlow
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# Load the MNIST dataset
mnist = tf.keras.datasets.mnist

# Load the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

table = [
    ["Training Data", x_train.shape, y_train.shape],
    ["Testing Data", x_test.shape, y_test.shape]
]

# Print the table
print(tabulate(table, headers=["Dataset", "X Shape", "Y Shape"], tablefmt="pretty"))

# Change the type to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Convert data to 3 channels
x_train = np.stack((x_train,), axis=-1)
x_test = np.stack((x_test,), axis=-1)

# Split the data into training, validation, and test sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Prepare the data for tabulate
table = [
    ["Training Data", x_train.shape, y_train.shape],
    ["Validation Data", x_val.shape, y_val.shape],
    ["Testing Data", x_test.shape, y_test.shape]
]

# Print the table
print(tabulate(table, headers=["Dataset", "X Shape", "Y Shape"], tablefmt="pretty"))

# Suppress all logs except errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

# Define image data generators
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

val_generator = ImageDataGenerator(rescale=1./255)

# Create iterators
train_iterator = train_generator.flow(x_train, y_train, batch_size=512, shuffle=True)
val_iterator = val_generator.flow(x_val, y_val, batch_size=512, shuffle=False)

# Define the batch size
batch_size = 512

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

# Suppress all logs except errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Create the model
model = Sequential()

# Add the pretrained ResNet50 model
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

# Add fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Freeze ResNet layers
model.layers[0].trainable = False

# Print model summary
model.summary()

# Check available device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_info = "GPU acceleration in place powered by nVIDIA (CUDA)"
elif torch.cuda.is_available() and torch.backends.mps.is_available():
    device = torch.device("mps")
    device_info = "GPU acceleration in place powered by Apple's Metal Performance Shaders (MPS)"
else:
    device = torch.device("cpu")
    device_info = "Using CPU... Best of luck..."

print("Training the Model")
print(device_info)

# Load pre-trained ResNet18 model
resnet = models.resnet18(pretrained=True)

# Modify last layer for CIFAR-10 (10 classes)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)

# Transfer learning setup
resnet = resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training parameters
num_epochs = 10
train_losses = []
val_losses = []
train_accs = []
val_accs = []

# Ensure train_iterator and val_iterator are defined correctly
# Replace ... with your actual data iterators for train and validation
train_iterator = train_generator.flow(x_train, y_train, batch_size=512, shuffle=True)
val_iterator = val_generator.flow(x_val, y_val, batch_size=512, shuffle=False)

# Training loop for transfer learning
for epoch in range(num_epochs):
    resnet.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    progress_bar = tqdm(enumerate(train_iterator), total=len(train_iterator), desc=f"Epoch {epoch+1}/{num_epochs}", unit=" batch")
    
    for i, (inputs, labels) in progress_bar:
        # Convert inputs and labels to PyTorch tensors
        inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
        labels = torch.tensor(labels, dtype=torch.long).to(device)
    
        # Debugging: Print input shape
        print(inputs.shape)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)


+---------------+-----------------+----------+
|    Dataset    |     X Shape     | Y Shape  |
+---------------+-----------------+----------+
| Training Data | (60000, 28, 28) | (60000,) |
| Testing Data  | (10000, 28, 28) | (10000,) |
+---------------+-----------------+----------+
+-----------------+--------------------+-------------+
|     Dataset     |      X Shape       |   Y Shape   |
+-----------------+--------------------+-------------+
|  Training Data  | (48000, 28, 28, 1) | (48000, 10) |
| Validation Data | (12000, 28, 28, 1) | (12000, 10) |
|  Testing Data   | (10000, 28, 28, 1) | (10000, 10) |
+-----------------+--------------------+-------------+


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

Training the Model
Using CPU... Best of luck...


Epoch 1/10:   0%|          | 0/94 [00:00<?, ? batch/s]

torch.Size([512, 28, 28, 1])


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[512, 28, 28, 1] to have 3 channels, but got 28 channels instead

In [17]:
# Suppress all logs except errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

# Define image data generators
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)

val_generator = ImageDataGenerator(rescale=1./255)

# Create iterators
train_iterator = train_generator.flow(x_train, y_train, batch_size=512, shuffle=True)
val_iterator = val_generator.flow(x_val, y_val, batch_size=512, shuffle=False)

# Define the batch size
batch_size = 512

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

# Suppress all logs except errors
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Create the model
model = Sequential()

# Add the pretrained ResNet50 model
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

# Add fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Freeze ResNet layers
model.layers[0].trainable = False

# Print model summary
model.summary()

# Check available device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    device_info = "GPU acceleration in place powered by nVIDIA (CUDA)"
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    device_info = "GPU acceleration in place powered by Apple's Metal Performance Shaders (MPS)"
else:
    device = torch.device("cpu")
    device_info = "Using CPU... Best of luck..."

print("Training the Model")
print(device_info)

# Load pre-trained ResNet18 model
resnet = models.resnet18(pretrained=True)

# Modify last layer for CIFAR-10 (10 classes)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)

# Transfer learning setup
resnet = resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training parameters
num_epochs = 10
train_losses = []
val_losses = []
train_accs = []
val_accs = []

# Ensure train_iterator and val_iterator are defined correctly
# Replace ... with your actual data iterators for train and validation
train_iterator = train_generator.flow(x_train, y_train, batch_size=512, shuffle=True)
val_iterator = val_generator.flow(x_val, y_val, batch_size=512, shuffle=False)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

Training the Model
GPU acceleration in place powered by Apple's Metal Performance Shaders (MPS)


/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [27]:
# Training loop for transfer learning
for epoch in range(num_epochs):
    resnet.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    progress_bar = tqdm(enumerate(train_iterator), total=len(train_iterator), desc=f"Epoch {epoch+1}/{num_epochs}", unit=" batch")
    
    for i, (inputs, labels) in progress_bar:
        # Convert inputs and labels to PyTorch tensors
        inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
        labels = torch.tensor(labels, dtype=torch.long).to(device)
    
        # Print input shape (optional)
        print(inputs.shape)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss and calculate accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Print running loss every 200 batches
        if i % 200 == 199:
            progress_bar.set_postfix({'loss': running_loss / 200})
            running_loss = 0.0


Epoch 1/10:   0%|          | 0/94 [00:00<?, ? batch/s]

torch.Size([512, 28, 28, 3])


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[512, 28, 28, 3] to have 3 channels, but got 28 channels instead

In [ ]:
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss and calculate accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Print running loss every 200 batches
        if i % 200 == 199:
            progress_bar.set_postfix({'loss': running_loss / 200})
            running_loss = 0.0

In [ ]:
    # Calculate training accuracy and loss
    train_loss = running_loss / len(train_iterator)
    train_acc = correct_train / total_train
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Validation
    resnet.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in val_iterator:
            # Convert inputs and labels to PyTorch tensors
            inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
            labels = torch.tensor(labels, dtype=torch.long).to(device)

            outputs = resnet(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_loss /= len(val_iterator)
    val_acc = correct_val / total_val
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # Print epoch summary
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print('Finished Transfer Learning Training')

In [ ]:
    # Print epoch summary
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print('Finished Transfer Learning Training')

In [ ]:
# Display statistics in a table
table_data = [["Epoch", "Train Loss", "Train Acc", "Val Loss", "Val Acc"]]
for i in range(num_epochs):
    table_data.append([i+1, train_losses[i], train_accs[i], val_losses[i], val_accs[i]])

print(tabulate(table_data, headers="firstrow", tablefmt="pretty"))

In [ ]:
# Save the final model (optional)
#torch.save(model.state_dict(), 'VGG16_model.pth')
#print('Model saved to VGG16_model.pth')

### 8.Model deployment
- Pick the best model
- Build an app using Flask - Can you host somewhere other than your laptop? +5 Bonus points if you use Tensorflow Serving
- User should be able to upload one or multiples images get predictions including probabilities for each prediction

#### Step 1: Train and Save the Best Model
- First, ensure you have a trained VGG16 model saved in a format suitable for TensorFlow Serving. Here’s an example of how to save the model:First, ensure you have a trained VGG16 model saved in a format suitable for TensorFlow Serving. Here’s an example of how to save the model:

`import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical`

`# Load CIFAR-10 dataset`
`(x_train, y_train), (x_test, y_test) = cifar10.load_data()`

`# Normalize the data`
`x_train = x_train.astype('float32') / 255.0`
`x_test = x_test.astype('float32') / 255.0`


`# One-hot encode the labels`
`y_train = to_categorical(y_train, 10)`
`y_test = to_categorical(y_test, 10)`

`# Load the VGG16 model without the top layer`
`base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))`

`# Freeze base model layers for fine-tuning`
`base_model.trainable = False`

`# Add custom top layers`
`x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)`  `# CIFAR-10 has 10 classes`

`# Create the model`
`model = Model(inputs=base_model.input, outputs=predictions)`

`# Compile the model with a lower learning rate for fine-tuning`
`model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])`

`# Train the model for a few epochs`
`model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test))`

`# Save the model in a format suitable for TensorFlow Serving`
`tf.saved_model.save(model, "path_to_saved_model")`
`

#### Step 2: Set Up TensorFlow Serving
Install TensorFlow Serving and run it to serve your model.

##### Install TensorFlow Serving:

On Debian-based systems
`echo "deb [trusted=yes] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && sudo apt update && sudo apt install tensorflow-model-server`

Start TensorFlow Serving:
`tensorflow_model_server --rest_api_port=8501 --model_name=my_model --model_base_path="path_to_saved_model"`

#### Step 3: Build a Flask Application
Create a Flask application to handle image uploads and make predictions using the served model.

In [ ]:
#Install Flask:
!pip install flask

#Create the Flask App (app.py):
import os
import requests
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import numpy as np
from PIL import Image

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def preprocess_image(image_path):
    image = Image.open(image_path).resize((32, 32))
    image = np.array(image).astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)
        
        # Preprocess the image
        image = preprocess_image(file_path)
        
        # Make a prediction
        url = 'http://localhost:8501/v1/models/my_model:predict'
        data = json.dumps({"instances": image.tolist()})
        headers = {"content-type": "application/json"}
        json_response = requests.post(url, data=data, headers=headers)
        predictions = json_response.json()['predictions'][0]
        
        # Return the predictions with probabilities
        return jsonify({'predictions': predictions})
    else:
        return jsonify({'error': 'File type not allowed'})

if __name__ == '__main__':
    if not os.path.exists(app.config['UPLOAD_FOLDER']):
        os.makedirs(app.config['UPLOAD_FOLDER'])
    app.run(host='0.0.0.0', port=5000)

#### Step 4: Run the Flask Application
Start your Flask application:

bash
`!python app.py`

#### Step 5: Upload Images and Get Predictions
You can use a tool like Postman or cURL to send an image to the Flask app and get predictions.

Example cURL Command:
`curl -X POST -F 'file=@path_to_image.jpg' http://localhost:5000/predict`

### This setup ensures your VGG16 model is served using TensorFlow Serving, and a Flask application allows users to upload images and receive predictions with probabilities.

# Tests and stuff

In [ ]:
# 5. Transfer learning


# Check if GPU or MPS (Apple Silicon) is available and set device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU: cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS: mps")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Load a pre-trained ResNet model
resnet = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Modify the last layer for CIFAR-10 (10 classes)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)

# Transfer learning training
resnet = resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)

# Training loop for transfer learning
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = resnet(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Transfer Learning Training')


In [15]:
#Model Deployment
#Deploy the best model using Flask and TensorFlow Serving. Create an app allowing users to upload images and get predictions.
#Flask App Example:
